# Neural Networks Optimized by Genetic Algorithm

Usually, feedforward neural networks are optimized by backpropagation method, to get the derivatives of weights and biases, and optimize them during each iteration of sotchastic gradient descending. However, there are alternatives for backpropagation methods, such as genetic algorithms which can search for optimal parameters using iterations of populations and generations.

The genetic algorithm can generate a large population of specified neural networks with different parameters, each neural network with a specific parameter combinations is an individual. Then through calculation of fitnesses, cross-over, mutations, the algorithm tries to evolve the population in terms of the fitness.

In [36]:
import numpy as np
from time import time
import matplotlib.pyplot as plt
import scipy.io as sio
%matplotlib inline

In [2]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# 1.Feedforward Neural Network

We need to compute feedforward propagation given input and weights, it is quite easy to initialize them. And with different parameters, we can initialize different neural networks.

We initialize weights and biases according to uniform distribution.

In [10]:
def initializeWeights(layers):
    if type(layers) is not list:
        print('Wrong input!')
        return None
    layers_num = len(layers)
    weights = []
    for l in range(layers_num-1):
        weight = np.random.uniform(low=-0.01, high=0.01, size=(layers[l], layers[l+1]))
        weights.append(weight)
    return weights

def initializeBiases(layers):
    if type(layers) is not list:
        print('Wrong input!')
        return None
    layers_num = len(layers)
    biases = []
    for l in range(layers_num-1):
        bias = np.random.uniform(low=-0.01, high=0.01, size=(layers[l+1]))
        biases.append(bias)
    return biases
        

In [106]:
class feedforwardnetwork:
    '''
    Args:
    input_data: input matrix,[batch_size X feature_num], array
    input_labels: one-hot labels, array
    weights: define the weights, list
    biases: define the biases, list
    '''
    def __init__(self, input_data, input_labels, weights, biases):
        self.input_data = input_data
        self.input_labels = input_labels
        self.weights = weights
        self.biases = biases
        
    #Define sigmoid function
    def sigmoid(self, Z):
        return 1/(1 + np.exp(-Z))
    
    #Calculate the accuracy
    def calAccuracy(self, y, y_test):
        '''Calculate Accuracy'''
        return np.mean(y==y_test)
    
    #Create a function to do predictions
    #Map a one-hot vector to a number
    def vec2num(self, data, label_num=10):
        '''Make predictions'''
        if len(data.shape) < 2:
            print('The input has too few dimensions')
            return None
        #Select the class which has largest probability
        predictions = [(z.argmax()+ 1)%label_num for z in data]    
        return np.array(predictions)
    
    def costFuncWithReg(self, h, lambda1=0.01):
        '''
        Calculate the cost of neural network
        Note here we use cross entropy
        Regularization is also taken into account
        '''
        y = self.input_labels
        if h is None or y is None or self.weights is None:
            print('Invalid Input!')
            return None
        sample_num = len(y)#Length of y
        #Cost of errors
        total = -np.mean(np.sum(y*np.log(h), axis=1))
        #Cost of regularization
        weights = np.array(self.weights)
        reg = 0
        for wgt in weights:
            reg += np.sum(wgt**2) * lambda1/2/sample_num
        total +=  reg    
        return total 
    
    def proceed(self):
        '''
        Finish the procedure of feed forward network 
        and calculate the output
        '''
        h, output_h, input_z = self.feedforwardNeuralNetwork()
        labels = self.vec2num(self.input_labels)
        predictions = self.vec2num(h)
        #self.fitness = self.costFuncWithReg(h)
        self.accuracy = self.calAccuracy(labels, predictions)
        self.fitness = self.accuracy
        
    def update(self, new_weights, new_biases):
        '''
        Update weights and biases
        '''
        self.weights = new_weights
        self.biases = new_biases
    
    #weights: weights for each layer, a list
    def feedforwardNeuralNetwork(self):
        '''Calculate feedforward propagation output'''
        ######Deal with extreme cases###
        X = self.input_data
        if X is None or self.weights is None:
            print('Invalid Input!')
            return None
        dim = X.shape
        if len(dim) < 2:
            print('X has too less variables')
            return None
        #####Define variables###########
        layer_num = len(self.weights)
        output_h = []#Output for each layer
        output_h.append(X)#The first layer is equal to input X
        input_z = []#Input for each layer, starts from the second layer
        #####Make alculations for each layer, except the input layer
        for i in range(layer_num):
            z = np.dot(output_h[i], self.weights[i])
            z += self.biases[i]
            h = self.sigmoid(z)
            output_h.append(h)
            input_z.append(z)
        return h, output_h, input_z    

In [107]:
layers = [784, 128, 10]
weights = initializeWeights(layers)
biases = initializeBiases(layers)

In [108]:
batch_images, batch_labels = mnist.train.images, mnist.train.labels

In [109]:
fw = feedforwardnetwork(batch_images, batch_labels, weights, biases)

In [110]:
start = time()
fw.proceed()
end = time()
print('Timing:{:.3f}'.format(end-start))

Timing:1.211


It takes much time to finish computation for each individual. So it must be time-consuming if we have a large population.

In [111]:
fw.fitness

0.098490909090909087

In [112]:
fw.update(weights, biases)

In [113]:
fw.proceed()

In [114]:
fw.fitness

0.098490909090909087

Now we have an feedforward object, we can create an instance of neural network by specifying the parameters. We can also create a population of neural networks with different parameters. How to generate the parameters, that's a question we will answer by genetic algorithm.

# 2.Create population

In this project, we can view the parameters of neural networks as genes, our goal is to find optimal genes, namely   parameters that make those neural networks performance well on predictions.
There are four major steps in a genetic algorithm:
- Create Population, randomly create parameters and generate neural networks according to the parameters
- Calculate fitness, calculate the fitness of each individual neural network
- Cross over, according to the fitness, select some individuals as parents, and cross over their genes to make children
- Mutate, select certain individual neural networks and make mutations on their parameters(genes)

Update the older generation during each iteration, for more information please visit: https://en.wikipedia.org/wiki/Genetic_algorithm .

## Create a group of parameters

In [208]:
class initPopulation:
    def __init__(self, layers, pop_size=100):
        '''
        Args:
        pop_size: number of population, integer
        layers: neuron number for each layer, list
        '''
        self.pop_size = pop_size
        self.layers = layers
        
    def __initializeWeights__(self):
        '''
        Initialize weights between each two neigbouring layers
        '''
        layers = self.layers
        if type(layers) is not list:
            print('Wrong input!')
            return None
        layers_num = len(layers)
        weights = []
        for l in range(layers_num-1):
            weight = np.random.uniform(low=-0.01, high=0.01, size=(layers[l], layers[l+1]))
            weights.append(weight)
        return weights

    def __initializeBiases__(self):
        '''
        Initialize biases for each layer(except for input layer)
        '''
        layers = self.layers
        if type(layers) is not list:
            print('Wrong input!')
            return None
        layers_num = len(layers)
        biases = []
        for l in range(layers_num-1):
            bias = np.random.uniform(low=-0.01, high=0.01, size=(layers[l+1]))
            biases.append(bias)
        return biases
    
    def generatePop(self):
        '''
        Generate a group of weights at random
        '''
        population = []
        for i in range(self.pop_size):
            weights = self.__initializeWeights__()
            biases = self.__initializeBiases__()
            #Initialize an individual
            individual = feedforwardnetwork(batch_images, batch_labels, weights, biases)
            #Calculate fitness
            individual.proceed()
            population.append(individual)
        return population

In [209]:
pop = initPopulation(layers)
population = pop.generatePop()

# 3.Genetic Algorithm

In this part, we are going to realize selection, crossover and mutation.

In [210]:
import  copy
class GA:
    def __init__(self, population):
        '''
        Initialize genetic algorithm
        '''
        self.population = population
        
    def select(self, ratio=0.3):
        '''
        Randomly select part of original population
        '''
        #Get the fitness for each individual
        fitnesses = [one.fitness for one in iter(self.population)]
        fitnesses.insert(0, 0)
        #Calculate the sum of the fitness
        total_fitness = np.sum(fitnesses)
        #Normalization
        fitnesses = np.array(fitnesses)/total_fitness
        #Accumulated sum
        probs = np.cumsum(fitnesses)
        select_num = int(ratio * len(self.population))
        select_pop_index = []
        #Select a parent according to its fitness value
        for _ in np.arange(select_num):
            p = np.random.uniform(0, 1)
            for i in np.arange(len(probs)-1):
                if (p<=probs[i+1]) & (p>probs[i]):
                    select_pop_index.append(i)
                    break
        return select_pop_index
        
    def crossover(self, parent1, parent2):
        '''
        Cross over the genes of parents and
        Generate children
        Args:
        parent1: individual object
        parent2: individual object
        '''
        weights1, weights2 = parent1.weights, parent2.weights
        biases1, biases2 = parent1.biases, parent2.biases
        new_weights1, new_weights2 = copy.deepcopy(weights1), copy.deepcopy(weights2)
        new_biases1, new_biases2 = copy.deepcopy(biases1), copy.deepcopy(biases2)
        weight_num = len(weights1)
        select_layer = np.random.choice(weight_num)
        weight_shape =  weights2[select_layer].shape
        row = np.random.choice(weight_shape[0])
        col = np.random.choice(weight_shape[1])
        #Swap the specified weights
        new_weights1[select_layer][row:, :col] = weights2[select_layer][row:, :col]
        new_weights2[select_layer][row:, :col] = weights1[select_layer][row:, :col]
        #Swap the specified biases
        biases_num = len(biases1)
        select_layer = np.random.choice(biases_num)
        #biase_shape =  len(biases2[selet_layer])
        #point = np.random.choice(biase_shape)
        new_biases1[select_layer] = biases2[select_layer]
        new_biases2[select_layer] = biases1[select_layer]
        #Update the weights and biases
        parent1.update(new_weights1, new_biases1)
        parent2.update(new_weights2, new_biases2)
        parent1.proceed()
        parent2.proceed()
        return parent1, parent2
    
    def mutation(self, individual):
        '''
        Mutation at several random point within an individual
        '''
        weights, biases = individual.weights, individual.biases
        #Mutate weights
        for i in np.arange(len(weights)):
            shape = weights[i].shape
            for _ in range(3):
                row = np.random.choice(shape[0])
                col = np.random.choice(shape[1])
                weights[i][row, col] = np.random.uniform(-0.01, 0.01)
        for i in np.arange(len(biases)):
            point = np.random.choice(len(biases[i]))
            biases[i][point] = np.random.uniform(-0.01, 0.01)
        individual.update(weights, biases)
        individual.proceed()
        return individual
        
    def proceed(self):
        '''
        Execute crossover and mutation
        '''
        select_pop_index = self.select()
        #Keep the best individual
        _, optimal_index = self.findMaximalIndividual()
        best_individual = self.population[optimal_index]
        #cross over
        pair_num = int(len(select_pop_index)/2)
        for i in np.arange(pair_num):
            parent1, parent2 = self.population[i], self.population[i+pair_num]
            child1, child2 = self.crossover(parent1, parent2)
            self.population[i], self.population[i+pair_num] = child1, child2
            
        #mutation
        for i in np.arange(3):
            index = np.random.choice(len(self.population))
            individual = self.population[index]
            self.population[index] = self.mutation(individual)
        #Keep the best individual
        self.population[optimal_index] = best_individual
        #return self.population
        
    def findMaximalIndividual(self):
        accuracies = np.array([one.accuracy for one in self.population])
        fitnesses = np.array([one.fitness for one in self.population])
        optimal_value = max(fitnesses)
        optimal_index = fitnesses.argmax()
        return optimal_value, optimal_index

In [205]:
ga = GA(population)

In [211]:
#Try 20 generations
for _ in range(100):
    ga.proceed()

In [212]:
ga.findMaximalIndividual()

(0.09916363636363637, 5)

The result is not satisfying as the population is not large and generations is not sufficient to search in the weights space. Backpropation method works much effective because they know how to find optimal direction. Perhaps io